概要：介绍tf.keras模块。

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,optimizers,losses,datasets

d:\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depr

使用softmax完成前向计算：

In [2]:
x = tf.constant([2., 1., 0.1])
layer = layers.Softmax(axis=-1)
layer(x)

<tf.Tensor: id=2, shape=(3,), dtype=float32, numpy=array([0.6590012 , 0.24243298, 0.09856589], dtype=float32)>

使用Sequential容器创建网络：

In [3]:
from tensorflow.keras import Sequential

In [4]:
network = Sequential([
    layers.Dense(3, activation=None),
    layers.ReLU(),
    layers.Dense(2, activation=None),
    layers.ReLU()
])
x = tf.random.normal([4, 3])
network(x)

<tf.Tensor: id=63, shape=(4, 2), dtype=float32, numpy=
array([[0.0281768 , 0.00191075],
       [0.01487637, 0.        ],
       [0.        , 0.        ],
       [0.        , 0.06323063]], dtype=float32)>

使用Sequential容器的add方法创建网络：

In [5]:
layers_num = 2
network = Sequential([])
for _ in range(layers_num):
    network.add(layers.Dense(3))
    network.add(layers.ReLU())
    
network.build(input_shape=(None, 4))
network.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              multiple                  15        
_________________________________________________________________
re_lu_2 (ReLU)               multiple                  0         
_________________________________________________________________
dense_3 (Dense)              multiple                  12        
_________________________________________________________________
re_lu_3 (ReLU)               multiple                  0         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________


In [6]:
x = tf.random.normal([4, 4])
network(x)

<tf.Tensor: id=140, shape=(4, 3), dtype=float32, numpy=
array([[0.        , 0.35182503, 0.        ],
       [0.        , 0.        , 0.34833193],
       [0.        , 0.4828511 , 0.        ],
       [0.        , 0.        , 0.3957541 ]], dtype=float32)>

打印待优化参数：

In [7]:
for p in network.trainable_variables:
    print(p.name, p.shape)

dense_2/kernel:0 (4, 3)
dense_2/bias:0 (3,)
dense_3/kernel:0 (3, 3)
dense_3/bias:0 (3,)


In [8]:
network = Sequential([layers.Dense(256, activation='relu'),
                     layers.Dense(128, activation='relu'),
                     layers.Dense(64, activation='relu'),
                     layers.Dense(32, activation='relu'),
                     layers.Dense(10)])
network.build(input_shape=(4, 28*28))
network.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              multiple                  200960    
_________________________________________________________________
dense_5 (Dense)              multiple                  32896     
_________________________________________________________________
dense_6 (Dense)              multiple                  8256      
_________________________________________________________________
dense_7 (Dense)              multiple                  2080      
_________________________________________________________________
dense_8 (Dense)              multiple                  330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________


In [9]:
for p in network.trainable_variables:
    print(p.name, p.shape)

dense_4/kernel:0 (784, 256)
dense_4/bias:0 (256,)
dense_5/kernel:0 (256, 128)
dense_5/bias:0 (128,)
dense_6/kernel:0 (128, 64)
dense_6/bias:0 (64,)
dense_7/kernel:0 (64, 32)
dense_7/bias:0 (32,)
dense_8/kernel:0 (32, 10)
dense_8/bias:0 (10,)


In [10]:
network.compile(optimizer=optimizers.Adam(learning_rate=0.01),
               loss=losses.CategoricalCrossentropy(from_logits=True),
               metrics=['accuracy'])

导入mnist数据集：

In [11]:
(x,y),(x_test,y_test) = datasets.mnist.load_data()

In [12]:
x = tf.convert_to_tensor(x, dtype=tf.float32)

In [13]:
x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)

In [14]:
x = x/255.
x_test = x_test/255.

In [15]:
y = tf.convert_to_tensor(y, dtype=tf.int32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.int32)

定义转换函数：

In [16]:
def proprocess(x, y):
    x = tf.reshape(x, [-1])
    return x, y

将张量转换为Dataset：

In [17]:
train_data = tf.data.Dataset.from_tensor_slices((x, y))
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [18]:
train_data = train_data.shuffle(1000).map(proprocess).batch(128)
test_data = test_data.shuffle(1000).map(proprocess).batch(128)

In [19]:
x,y = next(iter(train_data))
print(x.shape, y.shape)

(128, 784) (128,)


In [20]:
y[0:4]

<tf.Tensor: id=342, shape=(4,), dtype=int32, numpy=array([7, 7, 1, 3])>

In [21]:
history = network.fit(train_data, epochs=5, validation_data=test_data, validation_freq=2)

Epoch 1/5
469/469 [==============================] - 3s 7ms/step - loss: 7374095064634.3896 - accuracy: 0.0000e+00
Epoch 2/5
469/469 [==============================] - 3s 7ms/step - loss: 264602544161275.6875 - accuracy: 0.0000e+00 - val_loss: 669293314787976.1250 - val_accuracy: 0.0000e+00
Epoch 3/5
469/469 [==============================] - 3s 6ms/step - loss: 1532724182114184.5000 - accuracy: 0.0000e+00
Epoch 4/5
469/469 [==============================] - 3s 6ms/step - loss: 4414232966942172.0000 - accuracy: 0.0000e+00 - val_loss: 6577121390095503.0000 - val_accuracy: 0.0000e+00
Epoch 5/5
469/469 [==============================] - 3s 6ms/step - loss: 9235650745679356.0000 - accuracy: 0.0000e+00


In [22]:
history.history

{'loss': [7359615826694.512,
  264446172210187.47,
  1532324783088765.0,
  4413654165418619.5,
  9234777155291910.0],
 'accuracy': [0.0, 0.0, 0.0, 0.0, 0.0],
 'val_loss': [669293314787976.1, 6577121390095503.0],
 'val_accuracy': [0.0, 0.0]}

预测：

In [23]:
out = network.predict(x)

In [24]:
out.shape

(128, 10)

保存加载模型：

In [25]:
network.save_weights('weights.ckpt')

In [26]:
network.save('model.zx')

INFO:tensorflow:Assets written to: model.zx\assets


In [27]:
network.load_weights('weights.ckpt')

In [29]:
tf.keras.experimental.export_saved_model(network, 'network-mnist2')

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: ['eval']
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:S

In [30]:
network = tf.keras.experimental.load_from_saved_model('network-mnist')

自定义网络层：

In [31]:
class MyDense(layers.Layer):
    def __init__(self, in_dim, out_dim):
        super(MyDense, self).__init__()
        self.kernel = self.add_variable('w', [in_dim, out_dim], trainable=True)
        
    def call(self, inputs, training=None):
        out = inputs @ self.kernel
        out = tf.nn.relu(out)
        return out
        

In [32]:
net = MyDense(4, 3)
net.trainable_variables

[<tf.Variable 'w:0' shape=(4, 3) dtype=float32, numpy=
 array([[-0.00105202,  0.19985664, -0.43557054],
        [-0.8074279 , -0.24563181, -0.87064666],
        [ 0.82190156, -0.4405772 ,  0.55936456],
        [-0.06950831,  0.5598724 ,  0.60735273]], dtype=float32)>]

使用Sequential定义模型：

In [33]:
network = Sequential([
    MyDense(784, 256),
    MyDense(256, 128),
    MyDense(128, 64),
    MyDense(64, 32),
    MyDense(32, 10)
])

network.build(input_shape=(None, 784))
network.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
my_dense_1 (MyDense)         multiple                  200704    
_________________________________________________________________
my_dense_2 (MyDense)         multiple                  32768     
_________________________________________________________________
my_dense_3 (MyDense)         multiple                  8192      
_________________________________________________________________
my_dense_4 (MyDense)         multiple                  2048      
_________________________________________________________________
my_dense_5 (MyDense)         multiple                  320       
Total params: 244,032
Trainable params: 244,032
Non-trainable params: 0
_________________________________________________________________


自定义模型：

In [34]:
class MyModel(keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        
        self.fc1 = MyDense(784, 256)
        self.fc2 = MyDense(256, 128)
        self.fc3 = MyDense(128, 64)
        self.fc4 = MyDense(64, 32)
        self.fc5 = MyDense(32, 10)
        
    def call(self, inputs, training=None):
        x = self.fc1(inputs)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.fc5(x)
        return x

In [35]:
network = MyModel()

In [36]:
network.build(input_shape=(None, 28*28))
network.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
my_dense_6 (MyDense)         multiple                  200704    
_________________________________________________________________
my_dense_7 (MyDense)         multiple                  32768     
_________________________________________________________________
my_dense_8 (MyDense)         multiple                  8192      
_________________________________________________________________
my_dense_9 (MyDense)         multiple                  2048      
_________________________________________________________________
my_dense_10 (MyDense)        multiple                  320       
Total params: 244,032
Trainable params: 244,032
Non-trainable params: 0
_________________________________________________________________


模型乐园：

In [37]:
resnet = keras.applications.ResNet50(weights='imagenet', include_top=False)
resnet.summary()

d:\python36\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


A local file was found, but it seems to be incomplete or outdated because the md5 file hash does not match the original value of a268eb855778b3df3c7506639542a6af so we will re-download the data.
94658560/94653016 [==============================] - 104s 1us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
___________________________

新建池化层：

In [47]:
global_average = layers.GlobalAveragePooling2D()

In [48]:
x = tf.random.normal([4, 7, 7, 2048])
out = global_average(x)

In [49]:
out.shape

TensorShape([4, 2048])

新建一个全连接层：

In [51]:
fc = layers.Dense(10)

In [53]:
x = tf.random.normal([4, 2048])
out = fc(x)
out.shape

TensorShape([4, 10])

利用resnet组成一个新网络：

In [54]:
mynet = Sequential([resnet, global_average, fc])

测量器 keras.metrics：

平均误差：

In [57]:
losses_mean  = keras.metrics.Mean()

采样新数据：

In [59]:
losses_mean.update_state(float(3.4))

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=2.0>

In [62]:
losses_mean.update_state(float(5.4))

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=3.0>

采样多次后，获得结果：

In [63]:
losses_mean.result()

<tf.Tensor: id=34681, shape=(), dtype=float32, numpy=4.066667>

可视化：

In [66]:
monitor_file = tf.summary.create_file_writer('tf-monitor')

In [67]:
with monitor_file.as_default():
    tf.summary.scalar('train_loss', float(3.2),step=2)